# Segmenting and Clustering Neighborhoods in Toronto

## Introduction

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## 1. Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup # LIbrary to scrape websites

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 2. Scrape website and create dataframe

Build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [2]:
# Request Website
website = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
# Create soup object
soup = BeautifulSoup(website, 'lxml')
# Find wiki table
table = soup.find('table',{'class':'wikitable sortable'})
# Create empty lists for each of the table columns
PostCode = []
Borough = []
Neighborhood = []
# Fill lists with data
for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        PostCode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighborhood.append(cells[2].find(text=True).replace("\n", ""))

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.

In [3]:
# Create dataframe out of the lists
df=pd.DataFrame(PostCode,columns=['PostalCode'])
df['Borough'] = Borough
df['Neighborhood'] = Neighborhood

Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.

In [4]:
# Drop the cells with string "Not assigned"
df.drop(df[df['Borough']=="Not assigned"].index,inplace=True)

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that **M5A** is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma.

If a cell has a borough but a **Not assigned** neighborhood, then the neighborhood will be the same as the borough. 

In [5]:
# Merge Postal Code and Borough into one column
df['combined'] = df['PostalCode']+'+'+df['Borough']
# Use groupby method and a join lambda function to merge postal code areas with more than one neighborhood
Neighborhood1=df.groupby('combined')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).values
# Get Postal Codes
PostalCode1=df.groupby('combined')['Borough'].apply(list).index.tolist()
# Create new dataframe
dfnew=pd.DataFrame(PostalCode1,columns=['PostalCode'])
dfnew[['PostalCode','Borough']] = dfnew.PostalCode.str.split("+",expand=True,)
dfnew['Neighborhood'] = Neighborhood1
dfnew['Neighborhood'].replace('Not assigned',dfnew['Borough'][dfnew[dfnew['Neighborhood']=="Not assigned"].index.item()],inplace=True)
dfnew.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
dfnew.shape

(103, 3)